In [1]:
import pandas as pd
import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [95]:
df = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')
print(df.shape)
df.head()

(1154112, 7)


,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [96]:
df['duration'] = df['dropOff_datetime'] - df['pickup_datetime'] 
df['duration'] = df.duration.apply(lambda t: t.total_seconds() / 60.)
df['duration'].mean()

19.167224093791006

In [98]:
df_rm = df[(df.duration >= 1) & (df.duration <= 60)].copy()
print(df.shape[0] - df_rm.shape[0])
df_rm.shape

44286


(1109826, 8)

In [35]:
df_rm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1109826 entries, 0 to 1154111
Data columns (total 8 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   dispatching_base_num    1109826 non-null  object        
 1   pickup_datetime         1109826 non-null  datetime64[ns]
 2   dropOff_datetime        1109826 non-null  datetime64[ns]
 3   PUlocationID            182818 non-null   float64       
 4   DOlocationID            961919 non-null   float64       
 5   SR_Flag                 0 non-null        object        
 6   Affiliated_base_number  1109053 non-null  object        
 7   duration                1109826 non-null  float64       
dtypes: datetime64[ns](2), float64(3), object(3)
memory usage: 76.2+ MB


In [99]:
# fill nan with -1 
df_rm.fillna(-1, inplace=True)
df_rm.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1109826 entries, 0 to 1154111
Data columns (total 8 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   dispatching_base_num    1109826 non-null  object        
 1   pickup_datetime         1109826 non-null  datetime64[ns]
 2   dropOff_datetime        1109826 non-null  datetime64[ns]
 3   PUlocationID            1109826 non-null  float64       
 4   DOlocationID            1109826 non-null  float64       
 5   SR_Flag                 1109826 non-null  int64         
 6   Affiliated_base_number  1109826 non-null  object        
 7   duration                1109826 non-null  float64       
dtypes: datetime64[ns](2), float64(3), int64(1), object(2)
memory usage: 76.2+ MB


In [100]:
len(df_rm.loc[df_rm['PUlocationID'] == -1, 'PUlocationID']) * 100 / df_rm.shape[0]

83.52732770722618

In [101]:
df_rm.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1.0,-1.0,-1,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1.0,-1.0,-1,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1.0,72.0,-1,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1.0,61.0,-1,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1.0,71.0,-1,B00037,9.050000


In [102]:
categorical = ['PUlocationID', 'DOlocationID']
df_rm[categorical] = df_rm[categorical].astype(str)
df_rm


,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1.0,-1.0,-1,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1.0,-1.0,-1,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1.0,72.0,-1,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1.0,61.0,-1,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1.0,71.0,-1,B00037,9.050000
...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,-1,B03266,8.750000
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,-1,,57.600000
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,-1,B03285,16.200000
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,-1,B03285,19.433333


In [103]:
df_rm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1109826 entries, 0 to 1154111
Data columns (total 8 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   dispatching_base_num    1109826 non-null  object        
 1   pickup_datetime         1109826 non-null  datetime64[ns]
 2   dropOff_datetime        1109826 non-null  datetime64[ns]
 3   PUlocationID            1109826 non-null  object        
 4   DOlocationID            1109826 non-null  object        
 5   SR_Flag                 1109826 non-null  int64         
 6   Affiliated_base_number  1109826 non-null  object        
 7   duration                1109826 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(1), object(4)
memory usage: 76.2+ MB


In [104]:
train_dicts = df_rm[categorical].to_dict(orient='records')
train_dicts[1:4]

[{'PUlocationID': '-1.0', 'DOlocationID': '-1.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '72.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '61.0'}]

In [105]:
dv = DictVectorizer()
x_train = dv.fit_transform(train_dicts)
x_train

<1109826x525 sparse matrix of type '<class 'numpy.float64'>'
	with 2219652 stored elements in Compressed Sparse Row format>

In [106]:
y_train = df_rm.duration.values
y_train

array([17.        , 17.        ,  8.28333333, ..., 16.2       ,
       19.43333333, 36.        ])

In [107]:
lr = LinearRegression()
lr.fit(x_train, y_train)

LinearRegression()

In [108]:
y_pred = lr.predict(x_train)
mean_squared_error(y_train, y_pred, squared=False)



10.528519107210744

In [109]:
df1 = pd.read_parquet('./data/fhv_tripdata_2021-02.parquet')
df1['duration'] = df1['dropOff_datetime'] - df1['pickup_datetime'] 
df1['duration'] = df1.duration.apply(lambda t: t.total_seconds() / 60.)
df1['duration'].mean()

df1_rm = df1[(df1.duration >= 1) & (df1.duration <= 60)].copy()
df1_rm.fillna(-1, inplace=True)

categorical = ['PUlocationID', 'DOlocationID']
df1_rm[categorical] = df1_rm[categorical].astype(str)

val_dicts = df1_rm[categorical].to_dict(orient='records')
val_dicts[0:10]



[{'PUlocationID': '173.0', 'DOlocationID': '82.0'},
 {'PUlocationID': '173.0', 'DOlocationID': '56.0'},
 {'PUlocationID': '82.0', 'DOlocationID': '129.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '225.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '61.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '26.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '72.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '169.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '161.0'},
 {'PUlocationID': '13.0', 'DOlocationID': '182.0'}]

In [110]:
X_val = dv.transform(val_dicts)


In [111]:
y_val = df1_rm.duration.values
y_val

array([10.66666667, 14.56666667,  7.95      , ..., 25.38333333,
       18.05      , 16.        ])

In [112]:
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)



11.014283196111764